In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

class spotWrapper():
    def __init__(self):
        client_id = 'accd0aa479164ddcbf1cbf822512b80b'
        client_secret = '58bfc467435045e7b61c86fb03385729'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        self.spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    def getAlbumsTracks(self, id):
        return self.spot.album_tracks(id)
    def getAudioFeatures(self, song_id):
        return self.spot.audio_features(song_id)



In [7]:
#alb = spot.getAlbumsTracks("spotify:album:3MATDdrpHmQCmuOcozZjDa")["items"]#[2].keys()
spot = spotWrapper()

song_id, song_name, artist_uris, artist_names, explicit, preview_url = ([] for i in range(6))
        # search on album URI
album_uri = "spotify:album:3aITAVBURujVe8fhI2seeR"
album_data = spot.getAlbumsTracks(album_uri)["items"]       
number_of_artists = len(album_data[0]["artists"])
#print(number_of_artists)
#print(album_data.keys())
for num in range(0,number_of_artists):
    artist_name_in_list = album_data[0]["artists"][num]["name"]
    try:
        artist_uri_in_list = all_artists.loc[all_artists["spotify_name"] == artist_name_in_list, "artist_uri"].iloc[0]
    except:
        artist_uri_in_list = "0"
    artist_uris.append(artist_uri_in_list)
    artist_names.append(artist_name_in_list)
for i in range(0,len(album_data)):
    print(str(i))
    print(album_data[i]["name"])





0
Tic Tac
1
My Legacy
2
Heart In Pieces
3
Because of You
4
Bust a Move
5
Baby Sasuke
6
Stripes Like Burberry
7
Marni On Me
8
Sleeping On The Floor
9
Real Baby Pluto
10
Drankin N Smokin
11
Million Dollar Play
12
Plastic
13
That’s It
14
Bought A Bad Bitch
15
Rockstar Chainz
16
Lullaby
17
She Never Been To Pluto
18
Off Dat
19
I Don’t Wanna Break Up
20
Bankroll
21
Moment of Clarity
22
Patek
23
Over Your Head


In [33]:
spot_audio = spot.getAudioFeatures("spotify:track:0anUgBvnA4u5LsHUeLRprc")
spot_audio2 = spot.getAudioFeatures("0anUgBvnA4u5LsHUeLRprc")



In [34]:
spot_audio

[{'danceability': 0.942,
  'energy': 0.566,
  'key': 11,
  'loudness': -4.834,
  'mode': 1,
  'speechiness': 0.226,
  'acousticness': 0.0773,
  'instrumentalness': 0,
  'liveness': 0.11,
  'valence': 0.479,
  'tempo': 132.858,
  'type': 'audio_features',
  'id': '0anUgBvnA4u5LsHUeLRprc',
  'uri': 'spotify:track:0anUgBvnA4u5LsHUeLRprc',
  'track_href': 'https://api.spotify.com/v1/tracks/0anUgBvnA4u5LsHUeLRprc',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0anUgBvnA4u5LsHUeLRprc',
  'duration_ms': 189147,
  'time_signature': 4}]

In [35]:
spot_audio2

[{'danceability': 0.942,
  'energy': 0.566,
  'key': 11,
  'loudness': -4.834,
  'mode': 1,
  'speechiness': 0.226,
  'acousticness': 0.0773,
  'instrumentalness': 0,
  'liveness': 0.11,
  'valence': 0.479,
  'tempo': 132.858,
  'type': 'audio_features',
  'id': '0anUgBvnA4u5LsHUeLRprc',
  'uri': 'spotify:track:0anUgBvnA4u5LsHUeLRprc',
  'track_href': 'https://api.spotify.com/v1/tracks/0anUgBvnA4u5LsHUeLRprc',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0anUgBvnA4u5LsHUeLRprc',
  'duration_ms': 189147,
  'time_signature': 4}]